In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pickle
import seaborn as sns
from pandas import Timestamp
import gpflow
from gpflow.utilities import print_summary
from gpflow import set_trainable
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from math import sqrt
from datetime import timedelta
from gpflow.config import default_float
import math

In [3]:
def read_file(path):
    d = pickle.load(open(path, 'rb'))
    X = d['X']
    Y = d['Y']
    
    Y[Y==0] = np.nan
    keep = ~np.isnan(Y[:,0])
    Y = Y[keep,:]
    X = X[keep,:]
    
    return X, Y

In [4]:
f = lambda time: Timestamp.fromtimestamp(time*3600)
f2 = np.vectorize(f)

In [5]:
X, Y = read_file('kampala_23_09_20.p')

In [6]:
longs = [X[:,0][index] for index in sorted(np.unique(X[:,0], return_index=True)[1])]
lats = [X[:,1][index] for index in sorted(np.unique(X[:,1], return_index=True)[1])]

In [7]:
long, lat = longs[6], lats[6]
lat, long

(0.322108, 32.576)

In [8]:
location_indices = np.where(np.logical_and(X[:,0]==long, X[:,1]==lat))
Xtraining = X[np.logical_not(np.logical_and(X[:,0]==long, X[:,1]==lat))]
Ytraining = np.delete(Y, slice(location_indices[0][0],location_indices[0][-1]+1), axis=0)
Xtraining.shape, Ytraining.shape

((4770, 3), (4770, 1))

In [9]:
Xtest = X[np.logical_and(X[:,0]==long, X[:,1]==lat)]
Ytest = Y[location_indices[0][0]:location_indices[0][-1]+1]
Xtest.shape, Ytest.shape

((126, 3), (126, 1))

In [10]:
k = gpflow.kernels.RBF(lengthscales =[0.01, 0.01, 1]) + gpflow.kernels.Bias()
print_summary(k)

╒═════════════════════════════╤═══════════╤═════════════╤═════════╤═════════════╤═════════╤═════════╤══════════════════╕
│ name                        │ class     │ transform   │ prior   │ trainable   │ shape   │ dtype   │ value            │
╞═════════════════════════════╪═══════════╪═════════════╪═════════╪═════════════╪═════════╪═════════╪══════════════════╡
│ Sum.kernels[0].variance     │ Parameter │ Softplus    │         │ True        │ ()      │ float64 │ 1.0              │
├─────────────────────────────┼───────────┼─────────────┼─────────┼─────────────┼─────────┼─────────┼──────────────────┤
│ Sum.kernels[0].lengthscales │ Parameter │ Softplus    │         │ True        │ (3,)    │ float64 │ [0.01 0.01 1.  ] │
├─────────────────────────────┼───────────┼─────────────┼─────────┼─────────────┼─────────┼─────────┼──────────────────┤
│ Sum.kernels[1].variance     │ Parameter │ Softplus    │         │ True        │ ()      │ float64 │ 1.0              │
╘═════════════════════════════╧═

In [11]:
m = gpflow.models.SGPR(data=(Xtraining, Ytraining), kernel=k, mean_function=None, inducing_variable=Xtraining.copy())
m.likelihood.variance.assign(625)
print_summary(m)

╒═════════════════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═══════════╤═════════╤══════════════════════════════════════════════════╕
│ name                                │ class     │ transform        │ prior   │ trainable   │ shape     │ dtype   │ value                                            │
╞═════════════════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═══════════╪═════════╪══════════════════════════════════════════════════╡
│ SGPR.kernel.kernels[0].variance     │ Parameter │ Softplus         │         │ True        │ ()        │ float64 │ 1.0                                              │
├─────────────────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼───────────┼─────────┼──────────────────────────────────────────────────┤
│ SGPR.kernel.kernels[0].lengthscales │ Parameter │ Softplus         │         │ True        │ (3,)      │ float64 │ [0.01 0.01 1.  ]                           

In [12]:
opt = gpflow.optimizers.Scipy()
opt_logs = opt.minimize(m.training_loss, m.trainable_variables, options=dict(maxiter=100))

InvalidArgumentError:  Cholesky decomposition was not successful. The input might not be valid.
	 [[node Cholesky (defined at c:\users\user\anaconda3\envs\ox\lib\site-packages\gpflow\models\sgpr.py:172) ]] [Op:__inference__tf_eval_886]

Errors may have originated from an input operation.
Input Source operations connected to node Cholesky:
 add_14 (defined at c:\users\user\anaconda3\envs\ox\lib\site-packages\gpflow\covariances\kuus.py:12)

Function call stack:
_tf_eval
